In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import sequence, text

import os
import glob
import json
import rdkit
from tqdm import tqdm
import logging

2024-06-06 10:26:46.074236: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-06 10:26:46.608196: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
# Folder path
DATASET_DIR = './datasets/'
MAPPING_DIR = './mapping/'

# SMILES path
SMILES_FILENAME = '250k_rndm_zinc_drugs_clean' # do not include '.smi' extension

# MAPPING TABLE PATH
MAPPING_TABLE_DIR = MAPPING_DIR + SMILES_FILENAME
WORD_IDX_FILENAME = 'word_to_idx.json'
IDX_WORD_FILENAME = 'idx_to_word.json'

In [6]:
# Create mapping folder
if not os.path.isdir(MAPPING_TABLE_DIR):
    try:
        os.makedirs(MAPPING_TABLE_DIR)
    except OSError:
        logging.error("Creation of the directory failed")
    else:
        print("Successfully created the directory")
else:
    logging.warning("Directory alread exists")

In [7]:
# Tokenization
# note: when char_tokenizerl=True, then filters not working! so we need split \n
tokenizer = text.Tokenizer(char_level=True, lower=False)

# Load data
with open(DATASET_DIR + SMILES_FILENAME + '.smi', 'r') as f:
    for smiles in tqdm(f.readlines()):
        smiles = smiles.split('\n')[0]
        tokenizer.fit_on_texts(smiles)
        
    word_index = tokenizer.word_index
    print('Total number of SMILES: {}'.format(len(word_index)))

100%|██████████| 2/2 [00:00<00:00, 31655.12it/s]

Total number of SMILES: 2


In [8]:
# Add prefix
SMILES_DIM = len(word_index)
word_index['<START>'] = SMILES_DIM + 1 
word_index['<PAD>'] = SMILES_DIM + 2
word_index['<EOL>'] = SMILES_DIM + 3 

word_index

{'C': 1, 'c': 2, '<START>': 3, '<PAD>': 4, '<EOL>': 5}

In [9]:
# index to word
index_word = {index:word for word, index in word_index.items()}
index_word

{1: 'C', 2: 'c', 3: '<START>', 4: '<PAD>', 5: '<EOL>'}

In [10]:
# Save
## using tokenizer class method
tokenizer_json = tokenizer.to_json()
with open(MAPPING_TABLE_DIR + '/tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))

## old school|
with open(MAPPING_TABLE_DIR + '/' + WORD_IDX_FILENAME, 'w') as wi, \
    open(MAPPING_TABLE_DIR + '/' + IDX_WORD_FILENAME, 'w') as iw:
        wi.write(json.dumps(word_index))
        iw.write(json.dumps(index_word))